In [3]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER
import cftime
import datetime
from datetime import date
from matplotlib import pyplot
from matplotlib import colors
from matplotlib import font_manager
import matplotlib.colors as mcolors
from matplotlib.cm import ScalarMappable
import matplotlib.dates as mdates
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import matplotlib.ticker as mticker
from mpl_toolkits.mplot3d import Axes3D
import netCDF4
from netCDF4 import Dataset
import numpy
import os
import pandas
from PIL import Image
import random
import readline
import scipy
from scipy import fft
from scipy import linalg
from scipy import stats
from scipy.stats import poisson, ttest_ind
import seaborn
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import normalize
from statsmodels.tsa.ar_model import AutoReg
import xarray as xr

In [4]:
Diri = '/glade/u/home/whimkao//ExtraTrack/Output_Files/'

In [5]:
# Open File
def Open_File(File):
    DF = pandas.read_csv(File)
    DF = DF.drop("Unnamed: 0", axis=1)
    return (DF)

In [6]:
# Open Each File
def Files_Open(Model, Diri):
    Data_DF = Open_File(Diri+Model+'_Data_Output_V5.csv')
    ET_DF = Open_File(Diri+Model+'_ET_Output_V5.csv')
    Codes_DF = Open_File(Diri+Model+'_Codes_Output_V5.csv')
    Time = []
    Start_Time = []
    End_Time = []
# Edit Time Format
    for i in range(len(Data_DF)):
        Time.append(Datetime(Data_DF["Time(Z)"][i]))
    for j in range(len(ET_DF)):
        Start_Time.append(Datetime(ET_DF["Start Time"][j]))
        End_Time.append(Datetime(ET_DF["End Time"][j]))
    Data_DF["Time(Z)"] = Time
    ET_DF["Start Time"] = Start_Time
    ET_DF["End Time"] = End_Time
    return (Data_DF, ET_DF, Codes_DF)

In [7]:
def Datetime(Time):
    New_Time = datetime.datetime.strptime(Time, '%Y-%m-%d %H:%M:%S')
    return (New_Time)

In [8]:
Control_Data, Control_ET, Control_Codes = Files_Open("Control", Diri)
RCP45_Data, RCP45_ET, RCP45_Codes = Files_Open("RCP45", Diri)
RCP85_Data, RCP85_ET, RCP85_Codes = Files_Open("RCP85", Diri)

In [9]:
# Find a Specific Storm Within the DataFrame
def Find_Storm(DF, Code):
    DF_Storm = DF[DF["Code"] == Code].reset_index()
    return (DF_Storm)

In [10]:
# Create Function to Open Storm Composite Files
def Composite_File(File):
    Diri = '/glade/u/home/whimkao//ExtraTrack/Composite_Files_V5/'
    Compo_File = xr.open_dataset(Diri + File)
    return (Compo_File)

In [15]:
Control_A_Compo_nc = Composite_File('composite_dtime900.control.002.nc')
Control_A_Compo_nc

<xarray.Dataset>
Dimensions:        (x: 80, y: 80, snapshot: 10159)
Coordinates:
  * x              (x) float64 -11.85 -11.55 -11.25 -10.95 ... 11.25 11.55 11.85
  * y              (y) float64 -11.85 -11.55 -11.25 -10.95 ... 11.25 11.55 11.85
Dimensions without coordinates: snapshot
Data variables: (12/62)
    snap_pathid    (snapshot) int32 ...
    snap_lon       (snapshot) float64 ...
    snap_lat       (snapshot) float64 ...
    snap_time      (snapshot) datetime64[ns] ...
    snap_U850      (snapshot, y, x) float32 ...
    snap_U500      (snapshot, y, x) float32 ...
    ...             ...
    PRECL          (y, x) float32 ...
    FLUT           (y, x) float32 ...
    CLDTOT         (y, x) float32 ...
    TMQ            (y, x) float32 ...
    OMEGA850       (y, x) float32 ...
    OMEGA500       (y, x) float32 ...

In [32]:
# Create DataFrame With Lat Lon Time Data of the Composite Files
def Composite_DF(Compo_nc, ABC):
    Snap_Time = pandas.Series(Compo_nc.snap_time)
    Snap_Lon = pandas.Series(Compo_nc.snap_lon) - 360.
    Snap_Lat = pandas.Series(Compo_nc.snap_lat)
    Snap_PathID = pandas.Series(Compo_nc.snap_pathid)
    Index = numpy.arange(0,len(Snap_Time),1)
    ABC_List = []
    for m in range(len(Index)):
        ABC_List.append(ABC)
    Compo_DF = pandas.DataFrame({"Orig Index": Index, "ABC": ABC_List, \
    "Time": Snap_Time, "Lon": Snap_Lon, "Lat": Snap_Lat, "PathID": Snap_PathID})
    return (Compo_DF)

In [33]:
# Combine Composite DFs
#def Combine_Compo_DF(Compo_A, Compo_B, Compo_C):
def Combine_Compo_DF(Compo_A):
    Compo_DF = Composite_DF(Compo_A, "A")
#    Compo_DF_B = Composite_DF(Compo_B, "B")
#    Compo_DF_C = Composite_DF(Compo_C, "C")
#    Compo_DF = pandas.concat(Compo_DF_A, Compo_DF_B, Compo_DF_C)
    return (Compo_DF)

In [34]:
Control_Compo = Combine_Compo_DF(Control_A_Compo_nc)

In [35]:
Control_Compo

,Orig Index,ABC,Time,Lon,Lat,PathID
0,0,A,1985-06-19 12:00:00,-82.186882,27.990064,0
1,1,A,1985-06-19 18:00:00,-82.452632,27.772887,0
2,2,A,1985-06-20 00:00:00,-82.235585,27.515720,0
3,3,A,1985-06-20 06:00:00,-82.399724,27.381539,0
4,4,A,1985-06-20 12:00:00,-82.457452,28.308044,0
...,...,...,...,...,...,...
10154,10154,A,2014-11-11 06:00:00,-76.855888,26.367210,306
10155,10155,A,2014-11-11 12:00:00,-75.845996,27.609009,306
10156,10156,A,2014-11-11 18:00:00,-74.544918,29.194649,306
10157,10157,A,2014-11-12 00:00:00,-73.068531,32.058408,306


In [26]:
# Change Year of Data
def Reverse_Update_Year(New_Time, Year_Diff):
    Year_Orig = New_Time.year + Year_Diff
#    print (Year_Orig)
    Orig_Time = New_Time.replace(year=Year_Orig)
    return (Orig_Time)

In [53]:
# Create Function to Find Indexes of Composite Data For Selected Storm
def Find_Composite_Data(Code, Data_DF, Compo_DF, Year_Diff):
    DF_Storm = Find_Storm(Data_DF, Code)
    New_Time = DF_Storm["Time(Z)"]
    Lat = DF_Storm["Lat"]
    Lon = DF_Storm["Lon"]
    Compo_Indexes = numpy.zeros(len(New_Time))
    for i in range(len(New_Time)):
        Orig_Time = Reverse_Update_Year(New_Time[i], Year_Diff)
        Compo_Storm = Compo_DF[Compo_DF["Time"] == Orig_Time].reset_index()
        if len(Compo_Storm) == 0:
            Compo_Indexes[i] = -728
        elif len(Compo_Storm) == 1:
            Compo_Indexes[i] = Compo_Storm["Orig Index"][0]
        else:
            Compo_Storm_1 = Compo_Storm[(Compo_Storm["Lon"] >= Lon[i]-0.5) & (Compo_Storm["Lon"] <= Lon[i]+0.5)]
            if len(Compo_Storm_1) == 1:
                Compo_Indexes[i] = Compo_Storm_1.reset_index()["Orig Index"][0]
            else:
                Compo_Storm_2 = Compo_Storm_1[(Compo_Storm["Lat"] >= Lat[i]-0.5) & (Compo_Storm["Lat"] <= Lat[i]+0.5)]
                if len(Compo_Storm_1) == 1:
                    Compo_Indexes[i] = Compo_Storm_2.reset_index()["Orig Index"][0]
                else:
                    print ("Kinen")
    return (Compo_Indexes)

In [54]:
Year_Diffs = numpy.array([[1985-1900, 1985-1930, 1985-1960]])

In [56]:
Compo_Indexes = Find_Composite_Data("TC191304", Control_Data, Control_Compo, Year_Diffs[0][0])
print (Compo_Indexes)

[3939. 3940. 3941. 3942. 3943. 3944. 3945. 3946. 3947. 3948. 3949. 3950.
 3951. 3952. 3953. 3954. 3955. 3956. 3957. 3958. 3959. 3960. 3961. 3962.
 3963. 3964. 3965. 3966. 3967. 3968. 3969. 3970. 3971. 3972. 3973. 3974.
 3975. 3976. 3977. 3978. 3979. 3980. 3981. 3982. 3983. 3984. 3985. 3986.
 3987. 3988. 3989. 3990. 3991. 3992. 3993. 3994. 3995. 3996. 3997. 3998.
 3999. 4000. 4001. 4002.]


In [29]:
File_Open

<xarray.Dataset>
Dimensions:        (x: 80, y: 80, snapshot: 10159)
Coordinates:
  * x              (x) float64 -11.85 -11.55 -11.25 -10.95 ... 11.25 11.55 11.85
  * y              (y) float64 -11.85 -11.55 -11.25 -10.95 ... 11.25 11.55 11.85
Dimensions without coordinates: snapshot
Data variables: (12/62)
    snap_pathid    (snapshot) int32 ...
    snap_lon       (snapshot) float64 ...
    snap_lat       (snapshot) float64 ...
    snap_time      (snapshot) datetime64[ns] ...
    snap_U850      (snapshot, y, x) float32 ...
    snap_U500      (snapshot, y, x) float32 ...
    ...             ...
    PRECL          (y, x) float32 ...
    FLUT           (y, x) float32 ...
    CLDTOT         (y, x) float32 ...
    TMQ            (y, x) float32 ...
    OMEGA850       (y, x) float32 ...
    OMEGA500       (y, x) float32 ...

In [58]:
print (float(Control_A_Compo_nc.snap_U10[3958,38,38]))

<xarray.DataArray 'snap_U10' ()>
[1 values with dtype=float32]
Coordinates:
    x        float64 -0.45
    y        float64 -0.45